### 利用SVD简化数据
- 我们可以记录用户关于餐馆观点的数据进行处理，并且从中提取出其背后的因素。这些因素可能会与餐馆的类别、烹饪时所用的某个特定配料，或其他任意对象一致。然后我们可以利用这些因素来估计人们对没有去过的餐馆的看法。提取这些信息的方法称为奇异值分解（Singular Value Decomposition，SVD）.
- 优点：简化数据，去除噪声，提高算法的结果。
- 缺点：数据的转换可能难以理解。

#### SVD的应用
- 信息检索：称利用SVD的方法为隐性语义索引（Latent Semantic Indexing, LSI）或隐性语义分析（Latent Semantic Analysis，LSA）。在LSI中，一个矩阵由文档和词语组成。当我们在该矩阵上应用SVD时，就会构建出多个奇异值。这些奇异值代表了文档中的概念或主题，这一特点可以用于更高效的文档搜索。
- 推荐系统：简单版本的推荐系统能够计算项或人之间的相似度。更先进的方法则先利用SVD从数据中构建一个主题空间，然后再在该空间下计算其相似度。

#### 矩阵分解
- 最常见的一种矩阵分解技术就是SVD。
- SVD将原始的数据集$Data$分解成三个矩阵$U,Σ和V^T$，如果原始矩阵$Data$是m行n列，那么$U,Σ和V^T$就分别是m行m列，m行n列，n行n列。如下所示  
$$Data_{m×n} = U_{m×m}Σ_{m×n}V^T_{n×n}$$
- 矩阵$Σ$只有对角函数，其他函数均为零。并且它的对角函数是从大到小排列的。这些对角元素成为奇异值（Singular Value）。他们对应了原始数据集矩阵$Data$的奇异值。
- SVD得到的是矩阵的奇异值，而PCA得到的是矩阵的特征值，其实奇异值和特征值是有关系的。这里的奇异值就是矩阵$Data*Data^T$特征值的平方根。
- 在科学和工程中，一直存在这样一个普遍事实：在某个奇异值的数目（r个）之后，其他的奇异值都置为0。这就意味着数据集中仅有r个重要特征，而其他的特征则都是噪声或者冗余特征。

In [20]:
import numpy as np
U,Sigma,VT = np.linalg.svd([[1,1],[7,7]])
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [2]:
Sigma

array([  1.00000000e+01,   2.82797782e-16])

In [3]:
VT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [4]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [7]:
Data = loadExData()

In [8]:
U,Sigma,VT = np.linalg.svd(Data)

In [9]:
Sigma

array([  9.64365076e+00,   5.29150262e+00,   6.51609210e-16,
         2.14818942e-16,   5.18511491e-17])

In [11]:
Sig3=np.mat([[Sigma[0], 0, 0], [0, Sigma[1], 0], [0, 0, Sigma[2]]])

In [12]:
U[:, :3]*Sig3*VT[:3, :]

matrix([[ -4.37395897e-16,   1.04047305e-15,   1.22370145e-15,
           2.00000000e+00,   2.00000000e+00],
        [ -6.64717157e-16,   3.85916933e-16,   2.78800224e-16,
           3.00000000e+00,   3.00000000e+00],
        [ -1.99303530e-16,   1.22205100e-16,   7.70984294e-17,
           1.00000000e+00,   1.00000000e+00],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -7.27982527e-17,  -7.16286297e-17],
        [  2.00000000e+00,   2.00000000e+00,   2.00000000e+00,
          -1.15127656e-16,  -1.11630659e-16],
        [  5.00000000e+00,   5.00000000e+00,   5.00000000e+00,
           4.54084131e-17,   4.34259903e-17],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -5.75638278e-17,  -5.58153296e-17]])

#### 基于协同过滤的推荐引擎
- 有很多方法可以实现推荐功能，这里我们只使用一种称为协同过滤（collaborative filter）的方法。
- 我们不利用专家所给出的重要属性来描述物品从而计算它们之间的相似度，而是利用用户对它们的意见来计算相似度。这就是协同过滤中所使用的方法。
- 计算相似度的三种方法：欧式距离，相似度=1/(1+距离)；皮尔逊相关系数（Pearson correlation）；余弦相似度（cosine similarity）。

In [22]:
from numpy import linalg as la

def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*np.corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [15]:
myMat=np.mat(loadExData())
ecludSim(myMat[:, 0], myMat[:, 4])

0.12973190755680383

In [16]:
ecludSim(myMat[:, 0], myMat[:, 0])

1.0

In [17]:
cosSim(myMat[:, 0], myMat[:, 4])

0.5

In [18]:
cosSim(myMat[:, 0], myMat[:, 0])

1.0

In [23]:
pearsSim(myMat[:, 0], myMat[:, 4])

0.20596538173840329

In [24]:
pearsSim(myMat[:, 0], myMat[:, 0])

1.0

- 如果用户的数目很多，那么我们可能倾向于使用基于物品相似度的计算方法，而不采用基于用户相似度的方法。
- 对于大部分产品导向的推荐引擎而言，用户的数量往往大于物品的数量，即购买商品的用户数会多于出售的商品种类。
- 通常用于推荐引擎评价的指标是称为最小均方根误差（Root Mean Squared Error，RMSE）的指标，它首先计算均方误差的平均值然后取其平均值。

#### 示例：餐馆菜肴推荐引擎
推荐未尝过的菜肴：
- 推荐系统的工作过程是：给定一个用户，系统会为此用户返回N个最好的推荐菜。
需要做到： 
- 寻找用户没有评级的菜肴，即在用户-物品矩阵中的0值；
- 在用户没有评级的所有物品中，对每个物品预计一个可能的评级分数。
- 对这些物品的评分从高到低进行排序，返回前N个物品。

In [35]:
def standEst(dataMat, user, simMeas, item):
    '''
    计算在给定相似度计算方法的条件下，用户对物品的估计评分值。
    - dataMat：数据矩阵
    - user：用户编号
    - simMeas：相似度计算方法
    - item：物品编号
    '''
    n = np.shape(dataMat)[1]  #得到数据集中的物品数目
    simTotal = 0.0; ratSimTotal = 0.0   #初始化后面用于计算估计评分值的变量
    for j in range(n):    #遍历行中的每个物品
        userRating = dataMat[user,j]    #用户评分
        if userRating == 0: continue    #如果某个物品的评分为零，则意味着没有用户对该物品进行评分
        overLap = np.nonzero(np.logical_and(dataMat[:,item].A>0, \
                                      dataMat[:,j].A>0))[0]    #给出两个物品当中已经被评分的那个元素
        if len(overLap) == 0: similarity = 0    #如果两者没有任何重合元素，则相似度为0且中止本次循环。
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])    #如果存在重合的物品，则基于这些重合物品计算相似度。
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity    #相似度不断累加
        ratSimTotal += similarity * userRating    #每次计算还考虑相似度和当前用户评分的乘积
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal    #除以所有评分的综合，归一化，返回0到5之间的评分值，这些评分值则用于对预测值进行排序

def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    '''
    推荐引擎，产生最高的N个推荐结果。
    - dataMat：数据矩阵
    - user：用户编号
    - N：N个推荐结果
    - simMeas：相似度计算方法
    - estMethod：估计方法
    '''
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]    #给定的用户建立一个未评分的物品列表
    if len(unratedItems) == 0: return 'you rated everything'     #如果不存在未评分物品，那么退出函数
    itemScores = []
    for item in unratedItems:    #在所有未评分物品上进行循环
        estimatedScore = estMethod(dataMat, user, simMeas, item)    #调用estMethod来产生该物品的预测分
        itemScores.append((item, estimatedScore))    #该物品的编号和估计得分值会放在一个元素列表中
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]    #排序，并返回结果

In [26]:
myMat=np.mat(loadExData())
myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])

In [36]:
recommend(myMat, 2)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [37]:
recommend(myMat, 2, simMeas=ecludSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.309017
the 1 and 4 similarity is: 0.333333
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 0.500000
the 2 and 4 similarity is: 0.000000


[(2, 3.0), (1, 2.8266504712098603)]

In [38]:
recommend(myMat, 2, simMeas=pearsSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 1.000000
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0)]

In [39]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [42]:
U,Sigma,VT=la.svd(np.mat(loadExData2()))
myMat=np.mat(loadExData2())

In [43]:
Sigma

array([ 15.77075346,  11.40670395,  11.03044558,   4.84639758,
         3.09292055,   2.58097379,   1.00413543,   0.72817072,
         0.43800353,   0.22082113,   0.07367823])

In [44]:
Sig2=Sigma**2

In [45]:
sum(Sig2)*0.9

487.7999999999995

In [46]:
sum(Sig2[:2])

378.8295595113579

In [47]:
sum(Sig2[:3])

500.50028912757932

- 该值高于总能量的90%，因此我们可以将一个11维的矩阵转换成一个3维的矩阵。

In [54]:
def svdEst(dataMat, user, simMeas, item):
    '''
    对给定用户构建一个评分估计值
    '''
    n = np.shape(dataMat)[1]    #得到物品总数
    simTotal = 0.0; ratSimTotal = 0.0    #初始化
    U,Sigma,VT = la.svd(dataMat)    #进行了SVD分解
    Sig4 = np.mat(np.eye(4)*Sigma[:4]) #建立对角矩阵
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #构建转换后的物品
    for j in range(n):    #与上述standEst类似
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [55]:
recommend(myMat, 1, estMethod=svdEst)

the 0 and 3 similarity is: 0.490950
the 0 and 5 similarity is: 0.484274
the 0 and 10 similarity is: 0.512755
the 1 and 3 similarity is: 0.491294
the 1 and 5 similarity is: 0.481516
the 1 and 10 similarity is: 0.509709
the 2 and 3 similarity is: 0.491573
the 2 and 5 similarity is: 0.482346
the 2 and 10 similarity is: 0.510584
the 4 and 3 similarity is: 0.450495
the 4 and 5 similarity is: 0.506795
the 4 and 10 similarity is: 0.512896
the 6 and 3 similarity is: 0.743699
the 6 and 5 similarity is: 0.468366
the 6 and 10 similarity is: 0.439465
the 7 and 3 similarity is: 0.482175
the 7 and 5 similarity is: 0.494716
the 7 and 10 similarity is: 0.524970
the 8 and 3 similarity is: 0.491307
the 8 and 5 similarity is: 0.491228
the 8 and 10 similarity is: 0.520290
the 9 and 3 similarity is: 0.522379
the 9 and 5 similarity is: 0.496130
the 9 and 10 similarity is: 0.493617


[(4, 3.3447149384692283), (7, 3.3294020724526971), (9, 3.328100876390069)]

#### 基于SVD的图像压缩
- 原始的图像大小是32×32=1024像素，我们能否使用更少的像素来表示这张图呢？

In [68]:
def printMat(inMat, thresh=0.8):
    '''
    打印矩阵
    由于矩阵中包含浮点数，因此必须定义浅色和深色。
    当元素大于thresh时，打印1；小于阈值时，打印0
    '''
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k]) > thresh:
                print(1, end=''),
            else: print(0, end=''),
        print('') 

def imgCompress(numSV=3, thresh=0.8):
    '''
    图像压缩
    基于给定的奇异值数目来重构图像
    '''
    myl = []    #初始化一个列表
    for line in open('0_5.txt').readlines():    #打开文本文件，并从文件中以数值方式读入字符
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = np.mat(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    U,Sigma,VT = la.svd(myMat)
    SigRecon = np.mat(np.zeros((numSV, numSV)))
    for k in range(numSV):    #构建对角矩阵
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [69]:
imgCompress(2)

****original matrix******
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
00000000000111111